# Predicción de tópicos
Se desea extraer los tópicos principales de cada semana dentro del ámbito político

## Librerias

In [2]:
import pandas as pd

# Preprocesamiento

In [11]:
# Archivo npr.csv es para hacer pruebas

df = pd.read_csv('noticias-economicas-sample.csv')

textoPrueba = df[['text']]

print(textoPrueba.shape)
textoPrueba.head()

(10000, 1)


,text
0,Angela Merkel ayer en Berlin tras la cumbre te...
1,\n\n\n\n\nEnrique Quemada Clariana\nFusiones y...
2,¿Cuántas veces no ha querido ser su propio jef...
3,"El ministro de Sanidad, Salvador Illa, momento..."
4,El Pleno del Congreso ha aprobado el proyecto ...
